# Week 6 Code Explanation



In [4]:
# Importing necessary libraries
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import community as community_louvain


## Query 0: Preparing the Graph

We start by loading the graph data from CSV files, create an undirected, unweighted graph, and focus on its largest connected component.


In [5]:
# Load the graph data from CSV files
edges_filename = "edges.csv"
nodes_filename = "nodes.csv"

df_edges = pd.read_csv(edges_filename)
df_nodes = pd.read_csv(nodes_filename)

# Create the graph using the edges CSV
G = nx.from_pandas_edgelist(df_edges, '# source', ' target')

# Ensure the graph is undirected and unweighted
G = nx.to_undirected(G)

# Remove self-loops if necessary
if nx.is_frozen(G):
    G = nx.Graph(G)  # Unfreeze if frozen
G.remove_edges_from(nx.selfloop_edges(G))

# Keep only the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()


## Query 1: Implementing Community Detection Techniques

We implement three different community detection techniques: Bridge Removal Method, Modularity Optimization, and Label Propagation.


### A) Bridge Removal Partition

In [8]:
def bridge_removal_partition(G):
    # Create a copy of the graph to work with
    G_no_bridges = G.copy()

    # Identify all bridges in the graph
    bridges = list(nx.bridges(G_no_bridges))

    # Remove these bridges from the graph
    G_no_bridges.remove_edges_from(bridges)

    # Identify the connected components in the graph after bridge removal
    components = [G_no_bridges.subgraph(c).copy() for c in nx.connected_components(G_no_bridges)]

    # Initialize lists to store modularity scores and community partitions
    modularity_scores = []
    community_partitions = []

    # Iterate over each component formed after removing bridges
    for comp in components:
        # Get the nodes in the current component
        community = list(comp.nodes)

        # Identify all other nodes in the graph not in the current component
        other_nodes = set(G.nodes()) - set(community)

        # Create a partition dictionary where nodes in the community are marked '0' and others '1'
        partition = {node: 0 for node in community}
        partition.update({node: 1 for node in other_nodes})

        # Convert the partition dictionary to a format suitable for modularity calculation
        reverse_partition = {}
        for node, community_id in partition.items():
            reverse_partition.setdefault(community_id, set()).add(node)

        # Get the communities as a list of sets of nodes
        communities = list(reverse_partition.values())

        # Store the partition
        community_partitions.append(partition)

        # Calculate and store the modularity score for this partition
        modularity_score = nx.community.modularity(G, communities)
        modularity_scores.append(modularity_score)

    # Identify the partition with the highest modularity score
    max_modularity_index = modularity_scores.index(max(modularity_scores))

    # Return the partition with the highest modularity
    return community_partitions[max_modularity_index]

  
# Apply Bridge Removal
start_time = time.time()
best_partition_bridge_removal = bridge_removal_partition(G)
bridge_time = time.time() - start_time


### B) Modularity Optimization 

In [12]:
start_time = time.time()
partition_modularity_optimization = community_louvain.best_partition(G)
modularity_time = time.time() - start_time

### C) Label Propagation

In [13]:
start_time = time.time()
communities_label_prop = list(nx.community.label_propagation_communities(G))
label_prop_time = time.time() - start_time
partition_label_prop = {node: i for i, comm in enumerate(communities_label_prop) for node in comm}

## Query 2: Analysis and Comparison of Results

We collect the results from each method, including the number of clusters, cluster sizes, and modularity scores, and then compare these metrics across methods.


In [9]:
# Collecting results from each method
def collect_results(G, partition):
    # Convert the partition dictionary into a structure of sets, where each set represents a community
    reverse_partition = {}
    for node, community_id in partition.items():
        # For each node, add it to the set corresponding to its community
        reverse_partition.setdefault(community_id, set()).add(node)
    
    # Convert the sets of communities into a list
    communities = list(reverse_partition.values())

    # Count the number of communities (clusters) in the partition
    num_clusters = len(communities)

    # Calculate the size (number of nodes) of each community
    cluster_sizes = [len(c) for c in communities]

    # Compute the modularity of the partition, a measure of the strength of the division of a network into communities
    modularity = nx.community.modularity(G, communities)

    # Return the number of clusters, the size of each cluster, and the modularity score
    return num_clusters, cluster_sizes, modularity

bridge_results = collect_results(G, best_partition_bridge_removal)
modularity_results = collect_results(G, partition_modularity_optimization)
label_prop_results = collect_results(G, partition_label_prop)

# Creating a DataFrame for comparison
results_df = pd.DataFrame({
    'Method': ['Bridge Removal', 'Modularity Optimization', 'Label Propagation'],
    'Number of Clusters': [bridge_results[0], modularity_results[0], label_prop_results[0]],
    'Cluster Size Distribution': [bridge_results[1], modularity_results[1], label_prop_results[1]],
    'Computational Time (s)': [bridge_time, modularity_time, label_prop_time],
    'Modularity': [bridge_results[2], modularity_results[2], label_prop_results[2]]
})

print(results_df)


                    Method  Number of Clusters Cluster Size Distribution  \
0           Bridge Removal                   2                   [1, 81]   
1  Modularity Optimization                   6     [17, 32, 17, 6, 7, 3]   
2        Label Propagation                   2                   [5, 77]   

   Computational Time (s)  Modularity  
0                0.016789   -0.000019  
1                0.000000    0.427583  
2                0.007007    0.083829  


## Query 3: Analysis of Community Detection Methods

#### **Bridge Removal Method:**
- **Number of Clusters:** This method resulted in only 2 clusters, with a highly uneven distribution: a very small cluster (just 1 node) and a much larger one (81 nodes). This outcome implies that the removal of bridges in the network predominantly isolates a small subset of nodes, while the rest of the network remains largely interconnected.
- **Modularity:** The modularity score is extremely low, hovering around zero. Such a low score suggests that the community division offered by this method is almost arbitrary and does not reveal any significant structure within the network. It may oversimplify the community structure, failing to capture the complexity of the network.

#### **Modularity Optimization (Louvain Method):**
- **Number of Clusters:** This approach identified 6 clusters, displaying a more balanced size distribution ranging from 3 to 32 nodes. The discovery of a higher number of clusters with more varied sizes indicates that the network possesses a more intricate community structure.
- **Modularity:** The modularity score is considerably high at approximately 0.428. A high modularity score like this one indicates that the Louvain method successfully identified communities where nodes are more densely connected internally than with the rest of the network. This suggests that it is the most effective method in capturing the true community structure of your network.

#### **Label Propagation:**
- **Number of Clusters:** Similar to the Bridge Removal method, Label Propagation found 2 clusters but with a different distribution (5 and 77 nodes). This method, known for its simplicity and speed, appears to capture some aspects of the network's community structure but might not be as nuanced or accurate in complex networks.
- **Modularity:** The modularity score here, while better than that of the Bridge Removal method, is considerably lower than that achieved by the Modularity Optimization method. This indicates that while Label Propagation does recognize some community structure, it is not as effective or detailed as the Modularity Optimization approach.

### Summary and Conclusion

- **Modularity Optimization** (Louvain Method) emerges as the most effective method for this particular network. Its ability to identify a balanced and nuanced community structure, supported by a high modularity score, suggests that it captures the network's community dynamics most accurately.
- **Bridge Removal Method**, despite its conceptual simplicity, appears to be the least effective for this network. Its almost zero modularity score and binary community split indicate an oversimplification of the network's community structure.
- **Label Propagation** offers a middle ground with its simplicity and speed, but it does not capture the community structure as effectively as the Modularity Optimization method, particularly in networks with more complex structures.

In summary, the **Modularity Optimization** method appears to be the most effective for your network, given its higher modularity score and a more balanced community size distribution. The Bridge Removal method does not seem effective for your network, as indicated by its very low modularity. Label Propagation provides a middle ground, being faster but less accurate than Modularity Optimization.

## Query 4: Visualization Hints for Gephi

After choosing the best partition based on our analysis, we prepare the data for visualization in Gephi by exporting the network with community data.


In [11]:
# Choose the best partition based on your analysis
best_partition = partition_modularity_optimization

# Add community information to nodes for export
for node, comm_id in best_partition.items():
    G.nodes[node]['community'] = comm_id

# Export the graph with community data to GEXF for Gephi
nx.write_gexf(G, "network_with_communities.gexf")